In [1]:
import os
import numpy

In [2]:
from lsst.sims.catUtils.utils import ObservationMetaDataGenerator

/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


In [3]:
from lsst.utils import getPackageDir

In [4]:
# Specify the path to an OpSim database
dbPath = os.path.join(getPackageDir('sims_data'), 'OpSimData/opsimblitz1_1133_sqlite.db')

In [6]:
gen = ObservationMetaDataGenerator(database=dbPath, driver='sqlite')

In [7]:
bounds = [
        ('obsHistID',5973),
        ('expDate',1220779),
        ('fieldRA',numpy.degrees(1.370916)),
        ('fieldDec',numpy.degrees(-0.456238)),
        ('moonRA',numpy.degrees(2.914132)),
        ('moonDec',numpy.degrees(0.06305)),
        ('rotSkyPos',numpy.degrees(3.116656)),
        ('telescopeFilter','i'),
        ('rawSeeing',0.728562),
        ('seeing', 0.88911899999999999),
        ('sunAlt',numpy.degrees(-0.522905)),
        ('moonAlt',numpy.degrees(0.099096)),
        ('dist2Moon',numpy.degrees(1.570307)),
        ('moonPhase',52.2325),
        ('expMJD',49367.129396),
        ('altitude',numpy.degrees(0.781015)),
        ('azimuth',numpy.degrees(3.470077)),
        ('visitExpTime',30.0),
        ('airmass',1.420459),
        ('m5',22.815249),
        ('skyBrightness',19.017605)]

## Query the OpSim database

We can use these to query the Opsim database and get the OpSim records. These are in the form of `numpy.recarray`s

In [8]:
OpSimRecords = gen.getOpSimRecords(fieldRA=(numpy.degrees(1.370916), numpy.degrees(1.5348635)),
                                     limit=20)

On the other hand we can also get the instances of Observation Meta Data which are required to build CatSim and PhoSim Instance Catalogs

In [9]:
newresults = gen.getObservationMetaData(fieldRA=(numpy.degrees(1.370916), numpy.degrees(1.5348635)),
                                     limit=20)

However, if we just have OpSim records in the form of tuples, we can get instances of ObservationMetaData corresponding to them:

In [10]:
record = OpSimRecords[0]
colnames = OpSimRecords.dtype.names

In [11]:
print(record)

(11, 3157, 1.532509, -0.635345, 2.451784, 0.189578, 1.645979, 'y', 0.90396, 0.986894, -0.228723, -0.205508, 1.194487, 89.122626, 49353.036546, 0.910651, 1.960608, 30.0, 1.265982, 21.06111, 17.0)


In [12]:
print(colnames)

('obsHistID', 'expDate', 'fieldRA', 'fieldDec', 'moonRA', 'moonDec', 'rotSkyPos', 'filter', 'rawSeeing', 'finSeeing', 'sunAlt', 'moonAlt', 'dist2Moon', 'moonPhase', 'expMJD', 'altitude', 'azimuth', 'visitExpTime', 'airmass', 'fiveSigmaDepth', 'filtSkyBrightness')


In [13]:
obsMetaData = ObservationMetaDataGenerator.ObservationMetaDataFromPointing(record, 
                                                                          OpSimColumns=colnames,
                                                                        columnMap=ObservationMetaDataGenerator.OpSimColumnMap('finSeeing'))

In [14]:
obsMetaData.m5

{'y': 21.061109999999999}

In [15]:
obsMetaData.skyBrightness

17.0

## This can therefore be used with other ways of getting OpSim records

In [16]:
import pandas as pd

In [17]:
import sqlite3

In [18]:
conn = sqlite3.connect(dbPath)

In [19]:
results = pd.read_sql_query('SELECT * FROM Summary LIMIT 5', con=conn, index_col='obsHistID').to_records()

One problem in this is that string objects are represented as 'Object' in `pandas.DataFrame` as it does not have a string type. We can convert this in the following way

In [20]:
mytypes = []
for i, val in enumerate(results.dtype.descr):
    if val[1] == '|O':
        mytypes.append((val[0], '|S1'))
    else:
        mytypes.append(val)

In [21]:
results = results.astype(mytypes)

In [22]:
columnMap = ObservationMetaDataGenerator.OpSimColumnMap('finSeeing')

In [23]:
obs_metaData = ObservationMetaDataGenerator.ObservationMetaDataFromPointing(results[0], columnMap=columnMap, OpSimColumns=results.dtype.names)

In [24]:
obs_metaData.phoSimMetaData

OrderedDict([('Opsim_obshistid', (1, dtype('int64'))),
             ('SIM_SEED', (2771, dtype('int64'))),
             ('pointingRA', (1.6764829999999999, dtype('float64'))),
             ('pointingDec', (-1.082473, dtype('float64'))),
             ('Opsim_moonra', (2.4506920000000001, dtype('float64'))),
             ('Opsim_moondec', (0.18985099999999999, dtype('float64'))),
             ('Opsim_rotskypos', (1.280457, dtype('float64'))),
             ('Opsim_filter', ('y', dtype('S1'))),
             ('Opsim_rawseeing', (0.77972600000000003, dtype('float64'))),
             ('Opsim_sunalt', (-0.20924100000000001, dtype('float64'))),
             ('Opsim_moonalt', (-0.22903399999999999, dtype('float64'))),
             ('Opsim_dist2moon', (1.4073260000000001, dtype('float64'))),
             ('Opsim_moonphase', (89.153486000000001, dtype('float64'))),
             ('Opsim_expmjd', (49353.032078999997, dtype('float64'))),
             ('Opsim_altitude', (0.73819100000000004, dtype('flo

In [25]:
direct = gen.getObservationMetaDataOld(obsHistID=1)

In [26]:
direct[0].phoSimMetaData

OrderedDict([('Opsim_obshistid', (1, dtype('int64'))),
             ('SIM_SEED', (2771, dtype('int64'))),
             ('pointingRA', (1.6764829999999999, dtype('float64'))),
             ('pointingDec', (-1.082473, dtype('float64'))),
             ('Opsim_moonra', (2.4506920000000001, dtype('float64'))),
             ('Opsim_moondec', (0.18985099999999999, dtype('float64'))),
             ('Opsim_rotskypos', (1.280457, dtype('float64'))),
             ('Opsim_filter', ('y', dtype('S1'))),
             ('Opsim_rawseeing', (0.77972600000000003, dtype('float64'))),
             ('Opsim_sunalt', (-0.20924100000000001, dtype('float64'))),
             ('Opsim_moonalt', (-0.22903399999999999, dtype('float64'))),
             ('Opsim_dist2moon', (1.4073260000000001, dtype('float64'))),
             ('Opsim_moonphase', (89.153486000000001, dtype('float64'))),
             ('Opsim_expmjd', (49353.032078999997, dtype('float64'))),
             ('Opsim_altitude', (0.73819100000000004, dtype('flo

## Getting ObservationMetaData instances for OpSim records in `numpy.recarray`:

In [27]:
obs_metaDataList = list(ObservationMetaDataGenerator.ObservationMetaDataFromPointing(result, 
                                                                               columnMap=columnMap,
                                                                               OpSimColumns=results.dtype.names)
                        for result in results)

or directly 

In [28]:
obs_metaDataListDirect = ObservationMetaDataGenerator.ObservationMetaDataFromPointingArray(results)

In [29]:
obs_metaDataList == obs_metaDataList

True